#### code to seperate out users with threshold > 30

In [55]:
from fileinput import filename
import pandas as pd
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import svds
import random 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import scipy.stats as ss
import pickle

In [56]:
def ReadData():
    ml1m_dir = 'data/ratings.dat'
    ml1m_rating = pd.read_csv(ml1m_dir, sep='::', header=None, names=['uid', 'mid', 'rating', 'timestamp'],  engine='python')
    unique_uid = np.unique(np.array(ml1m_rating['uid'].tolist()))
    unique_mid = np.unique(np.array(ml1m_rating['mid'].tolist()))
    uid_dict = dict([(y,x) for x,y in enumerate(unique_uid)])
    mid_dict = dict([(y,x) for x,y in enumerate(unique_mid)])
    print('DICTIONARY PREPARED:')

    # init user item dictionary:
    
    uid_list = ml1m_rating['uid'].tolist()
    uid_list_len = len(uid_list)
    mid_list = ml1m_rating['mid'].tolist()
    mid_list_len = len(mid_list)
    rating_list = ml1m_rating['rating'].tolist()
    user_item_dict = {x:set() for x in range(len(unique_uid))}
    item_user_dict = {x:set() for x in range(len(unique_mid))}
    for i in range(uid_list_len):
        uid_list[i] = uid_dict[uid_list[i]]
        mid_list[i] = mid_dict[mid_list[i]]
        # rating_list[i] = 1 # comment this line if you want to activate explicit ratings
        user_item_dict[uid_list[i]].add(mid_list[i])
        item_user_dict[mid_list[i]].add(uid_list[i])
    tmp_df = pd.DataFrame({"uid":uid_list, "mid":mid_list, "ratings":rating_list})
    v = tmp_df.uid.value_counts()
    df = tmp_df[tmp_df.uid.isin(v.index[v.gt(30)])]
### code to store less than 30 interactions:
    df_less_30 = tmp_df[tmp_df.uid.isin(v.index[v.le(30)])]
    return df, df_less_30, len(np.unique(mid_list))

In [57]:
df_gt_30, df_le_30, unique_mids = ReadData()
print("GREATER THAN 30:\n", df_gt_30)
print("LESS THAN 30: \n", df_le_30)
print(len(df_gt_30))
print(len(df_le_30))
print("UNIQUE MIDS: ", unique_mids)

DICTIONARY PREPARED:
GREATER THAN 30:
           uid   mid  ratings
0           0  1104        5
1           0   639        3
2           0   853        3
3           0  3177        4
4           0  2162        5
...       ...   ...      ...
1000204  6039  1019        1
1000205  6039  1022        5
1000206  6039   548        5
1000207  6039  1024        4
1000208  6039  1025        4

[980300 rows x 3 columns]
LESS THAN 30: 
          uid   mid  ratings
233        3  3235        5
234        3  1120        3
235        3  2743        4
236        3  1124        4
237        3   971        4
...      ...   ...      ...
999740  6037  1288        2
999741  6037  2495        1
999742  6037  2511        3
999743  6037  3165        3
999744  6037  1007        5

[19909 rows x 3 columns]
980300
19909
UNIQUE MIDS:  3706


In [58]:
test_df = df_gt_30.groupby("uid").tail(1)
# print(len(df_gt_30))
train_df = df_gt_30.drop(df_gt_30.groupby('uid').tail(1).index, inplace=False)
assert(len(df_gt_30)== len(test_df) + len(train_df))
# print(len(test_df))
# print(len(train_df))
dic_train_df_uid_mapping = dict([(y,x) for x,y in enumerate(np.unique(train_df['uid']))])
dic_train_df_uid_rmapping = dict([(x,y) for x,y in enumerate(np.unique(train_df['uid']))])
### no need for mid mapping

uid_of_train_df = train_df['uid'].tolist()
for i in range(len(uid_of_train_df)):
    uid_of_train_df[i] = dic_train_df_uid_mapping[uid_of_train_df[i]]
# for index, row in train_df.iterrows():
#     train_df['uid'][index] = dic_train_df_uid_mapping[train_df['uid'][index]]
core_user_ko_input_train_df = pd.DataFrame({'uid':uid_of_train_df, 'mid':train_df['mid'], 'ratings':train_df['ratings']})

In [66]:
train_ui_dic = {}    
for user in range(6040):
    train_ui_dic[user] = []
for index,row in train_df.iterrows():
        train_ui_dic[row['uid']].append(row['mid'])

In [26]:
print(len(np.unique(train_df['mid'])))
# print(dic_train_df_uid_rmapping)

3699


In [59]:
def ExtractCoreUsers(dataframe, unique_user_len, unique_item_len):
    # print("# of rows in ml1m_ratings: ", len(dataframe))
    u_len = unique_user_len
    print("USER LEN:", u_len)
    # print(user_id)

    m_len = unique_item_len
    print("MOVIE LEN:", m_len)
    userItemMatrix = np.zeros(shape=(u_len, m_len))
    # print(userItemMatrix)

    for index, row in dataframe.iterrows():
        userItemMatrix[row['uid']][row['mid']] = row['ratings']
        # print(row['uid'], row['mid'])
    print("USER ITEM MATRIX: \n", userItemMatrix)

    df = pd.DataFrame(userItemMatrix)
    cosineSimilarity = cosine_similarity(df)
    print("SHAPE OF COSINE MATIX:\n ", cosineSimilarity.shape)

    listToStoreTopFiftyOfEveryUser = []
    for i in range(0, cosineSimilarity.shape[0]):
        idx = np.argpartition(cosineSimilarity[i], -50)[-50:]
        listToStoreTopFiftyOfEveryUser.append(idx)
    # print("Top fifty list: \n", listToStoreTopFiftyOfEveryUser)
    # listToStoreTopFiftyOfEveryUser = np.array(listToStoreTopFiftyOfEveryUser)
    flatten = np.concatenate(listToStoreTopFiftyOfEveryUser)
    listToStoreTopFiftyOfEveryUser = flatten.ravel()

    # print("List of top 50", listToStoreTopFiftyOfEveryUser)
    df = pd.DataFrame(listToStoreTopFiftyOfEveryUser)
    allUserList = df.value_counts().index.tolist()
    # print("ALL USERS LIST", allUserList)
    allUserList = list(sum(allUserList,()))
    # print("ALL USERS LIST", allUserList)
    twentyPercentUserList = allUserList[:int(len(allUserList)*0.2)]
    # print("TWENTY PERCENT USER:", len(twentyPercentUserList))
    # print("TWENTY PERCENT USER:", (twentyPercentUserList))
    coreusers = dataframe.iloc[np.where(dataframe.uid.isin(twentyPercentUserList))]
    # coreusers.reset_index()
    # print("CORE USERS:\n", coreusers)
    return coreusers

In [60]:
core_users = ExtractCoreUsers(core_user_ko_input_train_df, len(np.unique(uid_of_train_df)), unique_mids)
support_user_list = np.unique(core_users['uid'])
print("CORE USERS:" ,core_users)

USER LEN: 5231
MOVIE LEN: 3706
USER ITEM MATRIX: 
 [[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [3. 0. 0. ... 0. 0. 0.]]
SHAPE OF COSINE MATIX:
  (5231, 5231)
CORE USERS:           uid   mid  ratings
523         5   627        4
524         5   805        4
525         5  2708        5
526         5  3341        3
527         5  3550        3
...       ...   ...      ...
1000203  5230  1018        3
1000204  5230  1019        1
1000205  5230  1022        5
1000206  5230   548        5
1000207  5230  1024        4

[366403 rows x 3 columns]


In [61]:
print(train_df['uid'][999866])
print(train_df)

6038
          uid   mid  ratings
0           0  1104        5
1           0   639        3
2           0   853        3
3           0  3177        4
4           0  2162        5
...       ...   ...      ...
1000203  6039  1018        3
1000204  6039  1019        1
1000205  6039  1022        5
1000206  6039   548        5
1000207  6039  1024        4

[975069 rows x 3 columns]


In [62]:
# print("CORE USERS:" ,core_users)
# print("UNIQUE USERS:", len(np.unique(core_users['uid'])))
core_users_index_list = core_users.index.to_list()
non_core_user_index = (train_df.index.difference(core_users.index))
non_core_user_index = non_core_user_index.tolist()

core_users_df = train_df.loc[core_users_index_list]
non_core_user_df = train_df.loc[non_core_user_index]
print("NON CORE USERS:" ,non_core_user_df)
print("CORE USERS:" ,core_users)

NON CORE USERS:          uid   mid  ratings
0          0  1104        5
1          0   639        3
2          0   853        3
3          0  3177        4
4          0  2162        5
...      ...   ...      ...
999862  6038  1006        4
999863  6038  1009        4
999864  6038  1011        3
999865  6038  1014        4
999866  6038  1016        4

[608666 rows x 3 columns]
CORE USERS:           uid   mid  ratings
523         5   627        4
524         5   805        4
525         5  2708        5
526         5  3341        3
527         5  3550        3
...       ...   ...      ...
1000203  5230  1018        3
1000204  5230  1019        1
1000205  5230  1022        5
1000206  5230   548        5
1000207  5230  1024        4

[366403 rows x 3 columns]


In [63]:
print("TEST DF CONTAINS TEST FOR CORE AND NON CORE ENTITIES:\n" ,test_df)
# print(core_users['uid'])
unique_uids_in_support_trian = np.unique(np.array(core_users_df['uid']))
unique_uids_in_query_trian = np.unique(non_core_user_df['uid'])
print(len(unique_uids_in_support_trian))
support_test_df = test_df.loc[test_df['uid'].isin(unique_uids_in_support_trian)]
print("SUPPORT TEST DF:" ,support_test_df)
query_test_df = test_df.loc[test_df['uid'].isin(unique_uids_in_query_trian)]
print("QUERY TEST DF:\n", query_test_df)

TEST DF CONTAINS TEST FOR CORE AND NON CORE ENTITIES:
           uid   mid  ratings
52          0  1154        4
181         1  1155        5
232         2  1900        4
451         4   683        4
522         5    33        4
...       ...   ...      ...
998634   6034  1865        4
999522   6035  1867        3
999724   6036  1025        5
999867   6038  1025        4
1000208  6039  1025        4

[5231 rows x 3 columns]
1046
SUPPORT TEST DF:           uid   mid  ratings
553         6  3186        3
798         8   414        3
1199        9  1868        5
1467       12  1865        4
1693       14  1934        4
...       ...   ...      ...
997247   6020  1025        3
997889   6025  1025        5
998118   6029  1025        3
999724   6036  1025        5
1000208  6039  1025        4

[1046 rows x 3 columns]
QUERY TEST DF:
          uid   mid  ratings
52         0  1154        4
181        1  1155        5
232        2  1900        4
451        4   683        4
522        5    33   

In [67]:
support_train = []
for index,row in core_users_df.iterrows():
    support_train.append([row['uid'], row['mid'], row['ratings']])
query_train = []
for index, row in non_core_user_df.iterrows():
    query_train.append([row['uid'], row['mid'], row['ratings']])
support_test = []
for index, row in support_test_df.iterrows():
    support_test.append([row['uid'], row['mid'], row['ratings']])
query_test = []
for index, row in query_test_df.iterrows():
    query_test.append([row['uid'], row['mid'], row['ratings']])
user_his_dic = {}
for u in train_ui_dic.keys():
    user_his_dic[u] = train_ui_dic[u]